In [11]:
import pandas as pd 
# import requests
from dotenv import load_dotenv
import os 
import base64
from requests import post, get
import json
import psycopg2

In [2]:
#load the environment variables
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

#Test they have been loaded correctly
# print(client_id, client_secret)

In [3]:
# Uses client credentials to request access token
def get_token():
    auth_info = client_id + ":" + client_secret
    auth_bytes = auth_info.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    base_url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": "Basic " + auth_base64
    }
    data = {"grant_type": "client_credentials"}
    result = post(base_url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

token = get_token()
print(token)
    

BQAG6ZL8oe9IJjpzN8Wnl606ZMaP65LByo3d0bIjuVJsQohvf9ew8V3vioPeuei9AcVjY1we-u8Dc7THb0vidbDKeHrqkNbRMtAAT5fCotueEBbtrmQ


In [4]:
#Forms header for whenever we send requests
def get_header(token):
    return {"Authorization": "Bearer " + token}

In [5]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    
    query_url = url + query
    
    result = get(query_url, headers=headers)
    # print(f"Raw response for {artist_name}: {result.content}")  # Debugging line
    try:
        json_result = json.loads(result.content)
    except Exception as e:
        print(f"Error parsing JSON for {artist_name}: {e}")
        return None

    # Check for errors in the JSON response
    if "error" in json_result:
        print(f"API returned an error for {artist_name}: {json_result['error']}")
        return None
    
    # Extract the "artists" key
    json_result = json_result.get("artists", {}).get("items", [])
    
    if len(json_result) == 0:
        print(f"No artist with this name exists: {artist_name}")
        return None
    
    return json_result[0]
 

In [6]:
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

In [7]:
token = get_token()
result = search_for_artist(token, "Bruno-Mars")

# print(result)
artist_id = result["id"]

print(artist_id)
# print(result["name"])
songs = get_songs_by_artist(token, artist_id)
print(songs)

# for idx, song in enumerate(songs):
#     print(f"{idx + 1}. {song['name']}")

0du5cEVh5yTK9QJze8zA0C
[{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'}, 'href': 'https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms', 'id': '1HY2Jd0NmPuamShAr6KMms', 'name': 'Lady Gaga', 'type': 'artist', 'uri': 'spotify:artist:1HY2Jd0NmPuamShAr6KMms'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/0du5cEVh5yTK9QJze8zA0C'}, 'href': 'https://api.spotify.com/v1/artists/0du5cEVh5yTK9QJze8zA0C', 'id': '0du5cEVh5yTK9QJze8zA0C', 'name': 'Bruno Mars', 'type': 'artist', 'uri': 'spotify:artist:0du5cEVh5yTK9QJze8zA0C'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', '

In [8]:
# df = pd.DataFrame(songs)
# df2 = pd.DataFrame(result)
df1 = pd.DataFrame(songs)

# Select only the 'id', 'name', and 'popularity' columns
filtered_df = df1[['id', 'name', 'popularity', 'duration_ms']]

# Display the filtered DataFrame
filtered_df


,id,name,popularity,duration_ms
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,100,251667
1,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,90,169917
2,3w3y8KPTfNeOKPiqUTakBh,Locked out of Heaven,89,233478
3,47Slg6LuqLaX0VodpSCvPt,Just the Way You Are,81,220734
4,0KKkJNfGyhkQ5aFogxQAPU,That's What I Like,88,206693
5,0nJW01T7XtvILxQgC5J7Wh,When I Was Your Man,86,213826
6,1wVuPmvt6AWvTL5W2GJnzZ,Talking to the Moon,76,217866
7,4lLtanYk6tkMvooU0tWzG8,Grenade,80,222091
8,6SKwQghsR8AISlxhcwyA9R,Marry You,80,230192
9,0M3HkE321xpCbCYqVKzr1q,It Will Rain,80,257720


In [14]:
# List of artists
artists = ["Asake","Burna-Boy","Jungle","Kendrick-Lamar", "SZA","Dexta-Daps", "JID","Teddy-Swims", "Bruno-Mars", "Lady-Gaga", "Coldplay", "Taylor-Swift", "Bad-Bunny", "The-Weeknd", "Billie-Eilish", "Ariana-Grande", "Drake", "Rihanna", "Ed-Sheeran", "Sabrina-Carpenter", "Justin-Bieber", "Ariana-Grande", "Eminem", "Kanye-West", "Post-Malone", "BTS", "Travis-Scott"]

# Initialize an empty list to store data for all artists
all_songs_data = []

# Iterate over each artist in the list
for artist_name in artists:
    try:
        # Search for the artist
        result = search_for_artist(token, artist_name)
        if result is None:
            print(f"No data found for artist: {artist_name}")
            continue
        
        # Get the artist ID and name
        artist_id = result["id"]
        artist_name = result["name"]  # This extracts the correct name of the artist
        artist_popularity = result["popularity"]
        
        # Fetch the top tracks for the artist
        songs = get_songs_by_artist(token, artist_id)
        
        # Append relevant data from the songs to the list
        for song in songs:
            all_songs_data.append({
                "id": song["id"],
                "artist_name": artist_name,  # Add the artist's name to each song entry
                "artist_popularity": artist_popularity,
                "song_name": song["name"],
                "song_popularity": song["popularity"],
                "duration_ms": song["duration_ms"]
                
            })
    except Exception as e:
        print(f"An error occurred for artist {artist_name}: {e}")

# Convert the collected data into a DataFrame
final_df = pd.DataFrame(all_songs_data)

# Display the DataFrame
final_df

,id,artist_name,artist_popularity,song_name,song_popularity,duration_ms
0,2j8n2hpZlEMvtrjJ7n0ZIy,Asake,75,Bad Girl (feat. Asake),71,174253
1,13VXuHw3O8Yt7VwRIDqSo4,Asake,75,MMS,71,219628
2,5ZtK8XAVnoaGdBXZWCEVCY,Asake,75,Lonely At The Top,69,157714
3,1qxKva4IzWde7m2jLpZvDU,Asake,75,Active,65,172597
4,0Q1bMs3xLQiDEeaneehdxv,Asake,75,Remember,68,182142
...,...,...,...,...,...,...
265,5wG3HvLhF6Y5KTGlK0IW3J,Travis Scott,92,Trance (with Travis Scott & Young Thug),81,194786
266,2cYqizR4lgvp4Qu6IQ3qGN,Travis Scott,92,BUTTERFLY EFFECT,80,190557
267,4kjI1gwQZRKNDkw1nI475M,Travis Scott,92,MY EYES,81,251249
268,6wsqVwoiVH2kde4k4KKAFU,Travis Scott,92,I KNOW ?,80,211582


In [13]:
from sqlalchemy import create_engine

# Load environment variables from the .env file
load_dotenv()

# Database connection parameters
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")

# Create the connection string
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Write the DataFrame to the database in the 'student' schema
table_name = "ak_spotify"  # Desired table name
final_df.to_sql(
    table_name,
    engine,
    schema="student",
    if_exists="replace",  # Options: 'fail', 'replace', 'append'
    index=False  # Do not write the DataFrame index as a column
)

210